# Importing Packages

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy.special import jv
from scipy.integrate import quad  
from scipy.special import j0, j1
import tkinter as tk 
from tkintertable.Tables import TableCanvas
from tkintertable.TableModels import TableModel
from tkinter import ttk
from tkinter import filedialog 
#from compute import Calculation
from WuWanv02 import Calculation
#from compute_cython import Calculation
#from backcompute import Back_Calculation
from tkinter import *  
import cProfile
import pstats
# python setup.py build_ext --inplace
# python setup.py build_ext --inplace --compiler=mingw32

In [2]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import time
from tkintertable import TableCanvas, TableModel

class TableApp:
    def __init__(self, root):
        self.frame = ttk.Frame(root)
        self.frame.pack(fill=tk.BOTH, expand=True)

        self.model = TableModel()
        self.table = TableCanvas(self.frame, model=self.model, rowheight=30, grid_color = "black", linewidth = 2.5, rowselectedcolor = "light blue", cellbackgr = "#E84445")
        self.table.show()
        self.table.tablecolheader.grid_forget()
        self.table.tablerowheader.grid_forget()

        self.load_data()
        self.adjust_column_widths()
        self.set_cell_colors()
        self.custom_draw_merged_header()
        button_frame = ttk.Frame(root)
        button_frame.pack(pady=5)
        tk.Button(
            button_frame,
            text="Compute!",
            command=self.get_data,
            font=("Arial", 10, "bold"),
            bg="#696969",
            fg="black",
            padx=20,
            pady=5
        ).pack()

        self.result_text = tk.Text(root, height=3, width=200)
        self.result_text.pack(pady=10)

    def load_data(self):
        data = {
            'Layer':            ['Layered System', 'layer #', '1', '2', '3', '4', '5', '', '', '','Stress [MPa]'],
            'Modulus [MPa]':    ['Layered System', 'Modulus [MPa]', '4000', '400', '300', '200', '100', '', '', '','0.95'],
            'Poisson [-]':      ['Layered System', 'Poisson [-]', '0.30', '0.35', '0.35', '0.40', '0.40', '', '', '', 'Radius [mm]'],
            'Thickness [mm]':   ['Layered System', 'Thickness [mm]', '150', '240', '300', '500', 'semi-inf', '', '', '','150'],
            '':  ['', '', '', '', '', '', '', '', '', '', ''],
            'Evaluation point': ['Evaluation point #', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10' ],
            'r [mm]':           ['r [mm]', '0', '300', '600', '900', '1200', '1500', '1800', '2000', '3000', '4000'],
            'Deflection [μm]':  ['Deflection [μm]', '', '', '', '', '', '', '', '', '', ''],
        }
        df = pd.DataFrame(data)

        self.model.data = {}
        self.model.columnNames = []
        self.model.reclist = []
        self.model.importDict(df.to_dict(orient='index'))
        self.table.redraw()
    def custom_draw_merged_header(self):
        
        merged_cols = ['Layer', 'Modulus [MPa]', 'Poisson [-]', 'Thickness [mm]']
        
        
        if not hasattr(self, 'merged_label'):
            self.merged_label = tk.Label(
                self.frame, 
                text="Layered System", 
                font=("Arial", 10, "bold"),
                bg='#CCCCCC',
                relief='solid',
                borderwidth=1
            )
            
            total_width = sum(self.model.columnwidths.get(col, 100) for col in merged_cols)
            self.merged_label.place(x=4, y=4, width=total_width, height=30)
        if not hasattr(self, 'merged_label_2'):
            self.merged_label_2 = tk.Label(
                self.frame, 
                text="Loading", 
                font=("Arial", 10, "bold"),
                bg='#CCCCCC',  
                relief='solid',
                borderwidth=1
            )
            
            row_height = 30
            header_height = 30  
            y_position = header_height + (row_height * 8)  
            
            total_width = sum(self.model.columnwidths.get(col, 100) for col in merged_cols)
            self.merged_label_2.place(x=4, y=y_position + 4, width=total_width, height=row_height)
        if not hasattr(self, 'merged_label_3'):
            self.merged_label_2 = tk.Label(
                self.frame, 
                text="Evaluation point #", 
                font=("Arial", 10, "bold"),
                bg='#CCCCCC',  
                relief='solid',
                borderwidth=1
            )
            
            
            
            width = 490
            self.merged_label_2.place(x= 4 + width, y= 4, width=100, height=row_height)

        if not hasattr(self, 'merged_label_4'):
            self.merged_label_2 = tk.Label(
                self.frame, 
                text="r [mm]", 
                font=("Arial", 10, "bold"),
                bg='#CCCCCC',  
                relief='solid',
                borderwidth=1
            )
            
            
            
            width = 590
            self.merged_label_2.place(x= 4 + width, y= 4, width=80, height=row_height)

        if not hasattr(self, 'merged_label_3'):
            self.merged_label_2 = tk.Label(
                self.frame, 
                text="Deflection [μm]", 
                font=("Arial", 10, "bold"),
                bg='#CCCCCC',  
                relief='solid',
                borderwidth=1
            )
            
            
            
            width = 670
            self.merged_label_2.place(x= 4 + width, y= 4, width=100, height=row_height)

    #def adjust_column_widths(self):
        #for col in self.model.columnNames:
            #self.model.columnwidths[col] = max(len(col) * 10, 100)
        #self.table.redraw()
    def adjust_column_widths(self):
        column_widths = {
            'Layer': 100,
            'Modulus [MPa]': 110,
            'Poisson [-]': 90,
            'Thickness [mm]': 110,
            'Evaluation point': 100,
            'r [mm]': 80,
            'Deflection [μm]': 100,
        }
        
        for col_name, width in column_widths.items():
            if col_name in self.model.columnNames:
                self.model.columnwidths[col_name] = width
        
        for col in self.model.columnNames:
            if col not in column_widths:
                self.model.columnwidths[col] = max(len(col) * 8, 80)
        
        self.table.redraw()

    def col_idx(self, col_name):
        try:
            return self.model.columnNames.index(col_name)
        except ValueError:
            return None

    def color_cells(self, rows, cols, color):
        if rows is not None and not isinstance(rows, (list, tuple)):
            rows = [rows]
        if cols is not None and not isinstance(cols, (list, tuple)):
            cols = [cols]
        self.table.setcellColor(rows, cols, color, key='bg')

    def set_cell_colors(self):
        grey   = '#CCCCCC'   
        yellow = '#FFE0C1'  

        cols_text = {
            'Layer',
            'Load',
        }


        text_col_idx = [self.col_idx(c) for c in cols_text if self.col_idx(c) is not None]
        defl_col     = self.col_idx('Deflection [μm]')

                    
        self.color_cells(1, 0, grey) 
        self.color_cells(2, 0, grey) 
        self.color_cells(3, 0, grey)
        self.color_cells(4, 0, grey)
        self.color_cells(5, 0, grey) 
        self.color_cells(6, 0, grey)   
        self.color_cells(6, 3, grey)    
        self.color_cells(1, 1, grey)
        self.color_cells(1, 2, grey)
        self.color_cells(1, 3, grey)
        self.color_cells(10, 0, grey)
        self.color_cells(10, 2, grey)
        self.color_cells(1, 5, grey)
        self.color_cells(2, 5, grey)
        self.color_cells(3, 5, grey)
        self.color_cells(4, 5, grey)
        self.color_cells(5, 5, grey)
        self.color_cells(6, 5, grey)
        self.color_cells(7, 5, grey)
        self.color_cells(8, 5, grey)
        self.color_cells(9, 5, grey)
        self.color_cells(10, 5, grey)
        self.color_cells(2, 1, yellow) 
        self.color_cells(2, 2, yellow) 
        self.color_cells(2, 3, yellow) 
        self.color_cells(3, 1, yellow) 
        self.color_cells(3, 2, yellow) 
        self.color_cells(3, 3, yellow) 
        self.color_cells(4, 1, yellow) 
        self.color_cells(4, 2, yellow) 
        self.color_cells(4, 3, yellow)
        self.color_cells(5, 1, yellow) 
        self.color_cells(5, 2, yellow) 
        self.color_cells(5, 3, yellow)  
        self.color_cells(6, 1, yellow) 
        self.color_cells(6, 2, yellow) 
        self.color_cells(10, 1, yellow) 
        self.color_cells(10, 3, yellow) 
        self.color_cells(10, 6, yellow) 
        self.color_cells(9, 6, yellow)
        self.color_cells(8, 6, yellow)
        self.color_cells(7, 6, yellow) 
        self.color_cells(6, 6, yellow)
        self.color_cells(5, 6, yellow)
        self.color_cells(4, 6, yellow) 
        self.color_cells(3, 6, yellow)
        self.color_cells(2, 6, yellow)
        self.color_cells(1, 6, yellow)
       
        

        #if defl_col is not None:
            #rows_all = list(self.model.data.keys())
            #self.color_cells(rows_all, defl_col, result)

        self.table.redraw()


    def highlight_results(self):
        deflect_col = self.col_idx('Deflection [μm]')
        if deflect_col is None:
            return
        rows_to_hl = []
        for r in range(11):
            v = str(self.model.data.get(r, {}).get('Deflection [μm]', ''))
            if v not in ('', 'nan', 'None'):
                rows_to_hl.append(r)
        if rows_to_hl:
            self.color_cells(rows_to_hl, deflect_col, '#D6AFB9')  
        self.table.redraw()

    def get_data(self):
        df = pd.DataFrame.from_dict(self.model.data, orient='index')
        df.to_csv('data.csv', index=False)

        df_num = df.apply(pd.to_numeric, errors='coerce').fillna(0.0)
        arr = df_num.to_numpy()
        _, u1, u2, u3, u4, u5, u6, u7, u8, u9, u10 = Calculation(arr)
        values_mm  = [u1, u2, u3, u4, u5, u6, u7, u8, u9, u10]
        values_um  = [round(v * 1000, 1) for v in values_mm]

        df.loc[1:10, 'Deflection [mm]'] = values_mm
        df.loc[1:10, 'Deflection [μm]'] = values_um
        self.model.data = df.to_dict(orient='index')

        self.highlight_results()

        self.result_text.insert(tk.END,
                                f'Compute Finished! \n')
        print("Computation Finished!")

if __name__ == '__main__':
    root = tk.Tk()
    root.geometry('800x480')
    root.title('WuWan v0.2')

    app = TableApp(root)
    root.mainloop()


Run time: 0.001230125 s
Computation Finished!
